In [1]:
import numpy as np
import astropy.units as u 
from astropy.constants import c, L_sun
import matplotlib.pyplot as plt
from astropy.cosmology import Planck15 as cosmo
from scipy.interpolate import interp1d

#from Phi_Obs_hostgal_MiLim import get_phi_lam_obs
from phi_obs_general import get_phi_lam_obs
from qlfhosts.QLFs import S20_QLF as QLF

from qlfhosts.SEDs import R06_AGN, A10_hosts
from qlfhosts.AGN_Selection import R02
from qlfhosts.GLFs import Uniform, Willmer06, Kollmeier06

from qlfhosts.util.phiObs import PhiObs

In [2]:
z = 0.5
DL = cosmo.luminosity_distance(z)
lam_eff_filter = 4750.*u.angstrom
m_faint = 28.0
m_bright = 15.0
m_zp_jy = 3631.0*u.Jy


In [3]:
dmag = 0.5 * u.mag
m_grid = np.arange(m_bright, m_faint+0.1*dmag.value, dmag.value)
Ntot1 = np.zeros(m_grid.shape)
Ntot2 = np.zeros(m_grid.shape)
Ntot3 = np.zeros(m_grid.shape)

In [4]:
bp_names = ['sdssu', 'sdssg', 'sdssr']

#Set up the intrinsic qlf to use. 
qlf = QLF()

#Set up the AGN selection criteria.
sel_crit = R02()

agn_sed = R06_AGN(z, bp_names=bp_names, cosmo=cosmo)
hosts_sed = A10_hosts(z, bp_names=bp_names, cosmo=cosmo)
hosts_sed.likelihood = np.array([0. ,1. ,0.])

In [5]:
glf1 = Uniform()
glf2 = Willmer06(z=z)
glf3 = Kollmeier06()

In [6]:
lfact = np.log10(m_zp_jy * 4*np.pi*DL**2 * c/lam_eff_filter / qlf.Lstar_units)
lLlam_obs_min = lfact - 0.4*m_faint
lLlam_obs_max = lfact - 0.4*m_bright

In [7]:
#Estimate the observed QLF.
phi1, dlLlam1 = get_phi_lam_obs(z, qlf, lLlam_obs_min, lLlam_obs_max, lam_eff_filter , agn_sed, hosts_sed, sel_crit, glf1)
phi2, dlLlam2 = get_phi_lam_obs(z, qlf, lLlam_obs_min, lLlam_obs_max, lam_eff_filter , agn_sed, hosts_sed, sel_crit, glf2)
phi3, dlLlam3 = get_phi_lam_obs(z, qlf, lLlam_obs_min, lLlam_obs_max, lam_eff_filter , agn_sed, hosts_sed, sel_crit, glf3)

In [10]:
#Now, repeat the estimates but with the new module. 
phi1_obj = PhiObs(z, Galaxy_Luminosity_Distribution=Uniform)
phi1_new, dlLlam1_new = phi1_obj.get_phi_lam_obs(lLlam_obs_min, lLlam_obs_max, lam_eff_filter)

phi2_obj = PhiObs(z, Galaxy_Luminosity_Distribution=Willmer06)
phi2_new, dlLlam1_new = phi2_obj.get_phi_lam_obs(lLlam_obs_min, lLlam_obs_max, lam_eff_filter)

phi3_obj = PhiObs(z, Galaxy_Luminosity_Distribution=Kollmeier06)
phi3_new, dlLlam1_new = phi3_obj.get_phi_lam_obs(lLlam_obs_min, lLlam_obs_max, lam_eff_filter)

In [12]:
print(phi2_new[-10:-1])
print(phi2[-10:-1])

[3.17911528e-08 2.58207198e-08 2.09593832e-08 1.70041884e-08
 1.37885164e-08 1.11757971e-08 9.05424678e-09 7.33248396e-09
 5.93588990e-09] 1 / (dex Mpc3)
[3.17911528e-08 2.58207198e-08 2.09593832e-08 1.70041884e-08
 1.37885164e-08 1.11757971e-08 9.05424678e-09 7.33248396e-09
 5.93588990e-09] 1 / (dex Mpc3)
